In [1]:
# diagnosing rff masks

In [23]:
import xarray as xr
import numpy as np
mask_path = "/shares/gcp/integration/rff/climate/masks/CO2_Fossil"
# masks = {}

In [157]:
masks = {}

rff = xr.open_dataset(
    "/shares/gcp/integration/rff/climate/ar6_rff_iter0-19_fair162_CO2_Fossil_control_pulse_2020_temp_v5.02_newformat.nc"
).sel(year = 2300).squeeze(drop = True)

socioec = xr.open_dataset(
    "/shares/gcp/integration/rff/socioeconomics/rff_global_socioeconomics.nc4"
).sel(year=2300, region="world", drop=True)
gdppc = socioec.gdp / socioec.pop 
gdppc = gdppc.expand_dims({"simulation": range(0,20)})
emissions = (
    xr.open_dataset(
        "/shares/gcp/integration/rff/climate/rff-sp_emissions_all_gases.nc"
    )
    .emissions.sel(gas="C", drop=True)
    .sum("Year")
    .rename({"simulation": "rff_sp"})
).expand_dims({"simulation": range(0,20)})


for mask_name in ["climate","emissions","gdppc","gdppc_emissions","gdppc_emissions_climate"]:
    masks[mask_name] = xr.open_dataset(f"{mask_path}/{mask_name}_based_masks.nc4").squeeze(drop = True)
#     bring back in the simulation dim if it's not there
    try:
        masks[mask_name] = masks[mask_name].expand_dims({"simulation": range(0,20)})
    except:
        pass

#     count masked out runs
    for var in ["q0.001_q0.999","q0.005_q0.995","q0.01_q0.99","q0.05_q0.95"]: 
        current_mask = masks[mask_name][var]
        n_masked = 200000 - current_mask.sum().values
        print("Mask: ", mask_name, var)
        print("Runs masked: ", n_masked)
        masked_temp = rff.where(current_mask, drop = True)
        max_temp = float(masked_temp["control_temperature"].max(dim = ["rff_sp","simulation"]).values)
        min_temp = float(masked_temp["control_temperature"].min(dim = ["rff_sp","simulation"]).values)
        print("max temp: ", max_temp, "  min temp: ", min_temp)
        masked_emissions = emissions.where(current_mask, drop = True)
        max_emission = float(masked_emissions.max(dim = ["rff_sp","simulation"]).values)
        min_emission = float(masked_emissions.min(dim = ["rff_sp","simulation"]).values)
        print("max emission: ", max_emission, "  min emission: ", min_emission)
        masked_gdppc = gdppc.where(current_mask, drop = True)
        max_gdppc = float(masked_gdppc.max(dim = ["rff_sp","simulation"]).values)
        min_gdppc = float(masked_gdppc.min(dim = ["rff_sp","simulation"]).values)
        print("max gdppc: ", max_gdppc, "  min gdppc: ", min_gdppc)
        print()
        print()

        

Mask:  climate q0.001_q0.999
Runs masked:  400
max temp:  15.812644004821777   min temp:  -1.137488842010498
max emission:  10821.75541339544   min emission:  -2421.515476417515
max gdppc:  28266824522.995277   min gdppc:  679.5444331703698


Mask:  climate q0.005_q0.995
Runs masked:  2000
max temp:  12.313559532165527   min temp:  -0.5542527437210083
max emission:  10821.75541339544   min emission:  -2421.515476417515
max gdppc:  28266824522.995277   min gdppc:  679.5444331703698


Mask:  climate q0.01_q0.99
Runs masked:  4000
max temp:  11.107829093933105   min temp:  -0.34005287289619446
max emission:  10821.75541339544   min emission:  -2421.515476417515
max gdppc:  28266824522.995277   min gdppc:  679.5444331703698


Mask:  climate q0.05_q0.95
Runs masked:  20000
max temp:  8.575518608093262   min temp:  0.30238622426986694
max emission:  10243.820675229013   min emission:  -2421.515476417515
max gdppc:  28266824522.995277   min gdppc:  679.5444331703698


Mask:  emissions q0.001_